In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

# Load submission
df = pd.read_csv("submission.csv")

# Tree definition
trunk_w = 0.15
trunk_h = 0.2
base_w = 0.7
mid_w  = 0.4
top_w  = 0.25
tip_y = 0.8
tier_1_y = 0.5
tier_2_y = 0.25
base_y = 0.0
trunk_bottom_y = -trunk_h

BASE_COORDS = [
    (0.0, tip_y),
    (top_w / 2, tier_1_y),
    (top_w / 4, tier_1_y),
    (mid_w / 2, tier_2_y),
    (mid_w / 4, tier_2_y),
    (base_w / 2, base_y),
    (trunk_w / 2, base_y),
    (trunk_w / 2, trunk_bottom_y),
    (-(trunk_w / 2), trunk_bottom_y),
    (-(trunk_w / 2), base_y),
    (-(base_w / 2), base_y),
    (-(mid_w / 4), tier_2_y),
    (-(mid_w / 2), tier_2_y),
    (-(top_w / 4), tier_1_y),
    (-(top_w / 2), tier_1_y),
]

def get_poly(row):
    p = Polygon(BASE_COORDS)
    x = float(str(row['x']).replace('s', ''))
    y = float(str(row['y']).replace('s', ''))
    deg = float(str(row['deg']).replace('s', ''))
    p = affinity.rotate(p, deg, origin=(0,0))
    p = affinity.translate(p, x, y)
    return p

def plot_group(n):
    subset = df[df['id'].str.startswith(f"{n}_")]
    polys = []
    fig, ax = plt.subplots(figsize=(5,5))
    
    for _, row in subset.iterrows():
        p = get_poly(row)
        polys.append(p)
        x, y = p.exterior.xy
        ax.plot(x, y, 'b')
        ax.fill(x, y, 'b', alpha=0.3)
        
    u = unary_union(polys)
    minx, miny, maxx, maxy = u.bounds
    w = maxx - minx
    h = maxy - miny
    side = max(w, h)
    
    rect = plt.Rectangle((minx, miny), w, h, fill=False, color='r', linestyle='--')
    ax.add_patch(rect)
    
    # Draw square box
    sq_side = side
    rect_sq = plt.Rectangle((minx, miny), sq_side, sq_side, fill=False, color='g', linestyle=':')
    ax.add_patch(rect_sq)
    
    ax.set_title(f"N={n} Side={side:.4f} Eff={n*Polygon(BASE_COORDS).area/side**2:.2f}")
    ax.set_aspect('equal')
    plt.savefig(f"viz_N{n}.png")
    plt.close()

for n in [1, 2, 3, 4, 5]:
    plot_group(n)
